In [1]:
import sys
import numpy as np
import pandas as pd
import os 
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
#### 读取数据集 (只用训练集和测试集的销量数据)
path= 'data/Train/'
train_sales  = pd.read_csv(path+'train_sales_data.csv')
test_sales = pd.read_csv(path+'evaluation_public.csv')
submit_example = pd.read_csv(path+'submit_example.csv')

data = pd.concat([train_sales, test_sales], ignore_index=True)
data['label'] = np.log(data['salesVolume'])
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
for month in range(1,25): ## 16~17年销量
    tmp = data[data.mt==month][['adcode','model','label']]
    tmp.rename(columns={'label':month},inplace=True)
    data = data.merge(tmp)

In [3]:
old_model = ['3c974920a76ac9c1','3d7554f1f56dd664','2d0d2c3403909fdb',
 'a28bb927b6fcb33c','17bc272c93f19d56','2a2ab41f8f6ff1cb',
 'c06a2a387c0ee510','7023efdab9cedc03','af6f4f548684e14d',
 '7cf283430b3b5e38','d4efbebb087fd03f','7245e0ee27b195cd',
 '8c915fe4632fb9fa','6155b214590c66e6','28e29f2c03dcd84c',
 '37aa9169b575ef79','63065128401bb3ff','ea489c253676aafc',
 'cd5841d44fd7625e','b25c4e2e3856af22','4a103c30d593fbbe',
 '7a7885e2d7c00bcf','346393c2c6305fb1','02aab221aabc03b9',
 '5d7fb682edd0f937','a207df29ec9583f0','b4be3a4917289c82',
 'ef76a85c4b39f693','bb9fbec9a2833839','da457d15788fe8ee',
 '6858d6dfe680bdf7','79de4e4b24c35b04','12f8b7e14947c34d',
 '04e66e578f653ab9','dff803b4024d261d','61e73e32ad101892',
 'a432c483b5beb856','0797526c057dcf5b','936168bd4850913d',
 'cc21c7e91a3b5a0c','7aab7fca2470987e','fde95ea242abd896',
 '97f15de12cfabbd5','f5d69960089c3614','5b1c11c3efed5312',
 '17363f08d683d52b','06880909932890ca','9c1c7ee8ebdda299',
 'c6833cb891626c17','3e21824be728cbec','f8a6975573af1b33',
 '54fc07138d70374c','212083a9246d2fd3','4f79773e600518a6',
 'fc32b1a017b34efe','feabbf46658382b9','f270f6a489c6a9d7',
 'd0f245b8781e3631','c6cd4e0e073f5ac2','a9a43d1a7ecbe75d']

old_data_train = data.iloc[data[data.model.isin(old_model)].index]
new_data_train = data.iloc[data[~data.model.isin(old_model)].index]
print(old_data_train.index) # 旧数据在0~36959 
print(new_data_train.index) # 新数据在36960~50511 
old_data_test = test_sales.iloc[test_sales[test_sales.model.isin(old_model)].index]
new_data_test = test_sales.iloc[test_sales[~test_sales.model.isin(old_model)].index]

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            36950, 36951, 36952, 36953, 36954, 36955, 36956, 36957, 36958,
            36959],
           dtype='int64', length=36960)
Int64Index([36960, 36961, 36962, 36963, 36964, 36965, 36966, 36967, 36968,
            36969,
            ...
            50502, 50503, 50504, 50505, 50506, 50507, 50508, 50509, 50510,
            50511],
           dtype='int64', length=13552)


In [4]:
def get_shift_feat(df_):  #### 每种车型在各省16~17年的增长率 均值
    shift_feat = []
    df = df_.copy()
    model_adcode_2016_up = df[(df.regYear==2016) & (df.regMonth <= 6)].groupby(['model','adcode'])['label'].mean().reset_index()
    model_adcode_2017_up = df[(df.regYear==2017) & (df.regMonth <= 6)].groupby(['model','adcode'])['label'].mean().reset_index()
    model_adcode_2016_down = df[(df.regYear==2016) & (df.regMonth > 6)].groupby(['model','adcode'])['label'].mean().reset_index()
    model_adcode_2017_down = df[(df.regYear==2017) & (df.regMonth > 6)].groupby(['model','adcode'])['label'].mean().reset_index()
    model_adcode_2016_up.rename(columns={'label':'16mean_up'},inplace=True)
    model_adcode_2017_up.rename(columns={'label':'17mean_up'},inplace=True)
    model_adcode_2016_down.rename(columns={'label':'16mean_down'},inplace=True)
    model_adcode_2017_down.rename(columns={'label':'17mean_down'},inplace=True)
    df = df.merge(model_adcode_2016_up,on=['model','adcode'],how='left')
    df = df.merge(model_adcode_2017_up,on=['model','adcode'],how='left')
    df = df.merge(model_adcode_2016_down,on=['model','adcode'],how='left')
    df = df.merge(model_adcode_2017_down,on=['model','adcode'],how='left')
    df['factor_up'] = df['17mean_up'] / df['16mean_up']
    df['factor_down'] = df['17mean_down'] / df['16mean_down']
    return df

In [5]:
tmp_df = old_data_train[['adcode','model']].drop_duplicates()
print(tmp_df)
tmp_df2 = get_shift_feat(old_data_train)
df = pd.merge(tmp_df,tmp_df2,on=['adcode','model'],how='left')
df = df[['adcode','model']+list(range(1,25))+['16mean_up','17mean_up','16mean_down','17mean_down']+['factor_up','factor_down']].drop_duplicates()
df.loc[df.factor_up > 1.2,'factor_up'] = 1.01
df.loc[df.factor_up < 0.6,'factor_up'] = 0.99
df.loc[df.factor_down > 1.2,'factor_down'] = 1.01
df.loc[df.factor_down < 0.6,'factor_down'] = 0.99
### 指数平滑部分 
df[30] = (df[1]+df[2]+df[3])/3
ph_factor = 0.99
for i in range(1,25):  ### 一次平滑
    df[i+30] = df[i] * ph_factor + (1-ph_factor) * df[i+30-1] 
df[60] = (df[31]+df[32]+df[33])/3
for i in range(1,25):  ### 二次平滑
    df[i+60] = df[i+30] * ph_factor +(1-ph_factor) * df[i+60-1]
df[90] = (df[61]+df[62]+df[63])/3
for i in range(1,25):  ### 三次平滑
    df[i+90] = df[i+60] * ph_factor +(1-ph_factor) * df[i+90-1]
a = 3 * df[54] - 3 * df[84] + df[114]
b = ((6-5*ph_factor)*df[54]-2*(5-4*ph_factor)*df[84]+(4-3*ph_factor)*df[114]) * ph_factor / (2*(1-ph_factor)*(1-ph_factor))
c = (df[54]-2*df[84]+df[114]) * ph_factor*ph_factor/(2*(1-ph_factor)*(1-ph_factor))
for i in [25,26,27,28]:
    t = i - 24
    df[i] = a + b * t + c * t * t
###

temp_df = df[['adcode', 'model']].copy()
result_df = pd.DataFrame()
factor_up_fac = 0.4
for m in [25,26]:
    tmp1 = df[m-3]*0.2 + df[m-1]*0.3 +df[m-2]*0.2 + df[m]*0.3
    tmp2 = df[m-12]
    tmp3 = df[m-24]
    temp_df['forecastVolum'] = (tmp1 + tmp2 + tmp3)/3 * (factor_up_fac*df['factor_up']+(1-factor_up_fac)*df['factor_down']) * 0.99
    temp_df['regMonth'] = m-24
    if m==26:
        temp_df['forecastVolum'] = temp_df['forecastVolum']*0.99
    result_df = result_df.append(temp_df, ignore_index=True)
    df[m] = temp_df['forecastVolum']
for m in [27,28]:
    tmp1 = df[m-3]/6 + df[m-1]/2 + df[m-2]/6 + (df[m-12]*0.6+df[m-24]*0.4)/6
    tmp2 = df[m-12]/2 + df[m-13]/4 + df[m-11]/4
    tmp3 = df[m-24]
    temp_df['forecastVolum'] = (tmp1 + tmp2 + tmp3)/3 * (factor_up_fac*df['factor_up']+(1-factor_up_fac)*df['factor_down']) * 0.99
    temp_df['regMonth'] = m-24
    result_df = result_df.append(temp_df, ignore_index=True)
    df[m] = temp_df['forecastVolum']
old_data_test = pd.merge(old_data_test[['id', 'adcode', 'model', 'regMonth']], result_df, how='left', on=['adcode', 'model', 'regMonth'])
old_data_test['forecastVolum'] = np.exp(old_data_test['forecastVolum'])
print (old_data_test.groupby('regMonth')['forecastVolum'].mean())
old_data_test.loc[old_data_test['forecastVolum'] < 0, 'forecastVolum'] = 0

       adcode             model
0      310000  3c974920a76ac9c1
28     530000  3c974920a76ac9c1
56     150000  3c974920a76ac9c1
84     110000  3c974920a76ac9c1
112    510000  3c974920a76ac9c1
...       ...               ...
36820  350000  a9a43d1a7ecbe75d
36848  210000  a9a43d1a7ecbe75d
36876  500000  a9a43d1a7ecbe75d
36904  610000  a9a43d1a7ecbe75d
36932  230000  a9a43d1a7ecbe75d

[1320 rows x 2 columns]
regMonth
1    645.538082
2    416.927613
3    449.351544
4    455.090441
Name: forecastVolum, dtype: float64


In [6]:
tmp_df = new_data_train[['adcode','model']].drop_duplicates()
print(tmp_df)
tmp_df2 = get_shift_feat(new_data_train)
df = pd.merge(tmp_df,tmp_df2,on=['adcode','model'],how='left')
df = df[['adcode','model']+list(range(1,25))+['16mean_up','17mean_up','16mean_down','17mean_down']+['factor_up','factor_down']].drop_duplicates()
df.loc[df.factor_up > 1,'factor_up'] = np.sqrt(np.sqrt(df.loc[df.factor_up > 1,'factor_up']))
df.loc[df.factor_up < 0.7,'factor_up'] = np.sqrt(df.loc[df.factor_up < 0.7,'factor_up'])
df.loc[df.factor_down > 1,'factor_down'] = np.sqrt(np.sqrt(df.loc[df.factor_down > 1,'factor_down']))
df.loc[df.factor_down < 0.7,'factor_down'] = np.sqrt(df.loc[df.factor_down < 0.7,'factor_down'])

### 指数平滑部分 
df[30] = (df[1]+df[2]+df[3])/3
ph_factor = 0.99
for i in range(1,25):  ### 一次平滑
    df[i+30] = df[i] * ph_factor + (1-ph_factor) * df[i+30-1] 
df[60] = (df[31]+df[32]+df[33])/3
for i in range(1,25):  ### 二次平滑
    df[i+60] = df[i+30] * ph_factor +(1-ph_factor) * df[i+60-1]
df[90] = (df[61]+df[62]+df[63])/3
for i in range(1,25):  ### 三次平滑
    df[i+90] = df[i+60] * ph_factor +(1-ph_factor) * df[i+90-1]
a = 3 * df[54] - 3 * df[84] + df[114]
b = ((6-5*ph_factor)*df[54]-2*(5-4*ph_factor)*df[84]+(4-3*ph_factor)*df[114]) * ph_factor / (2*(1-ph_factor)*(1-ph_factor))
c = (df[54]-2*df[84]+df[114]) * ph_factor*ph_factor/(2*(1-ph_factor)*(1-ph_factor))
for i in [25,26,27,28]:
    t = i - 24
    df[i] = a + b * t + c * t * t
###

temp_df = df[['adcode', 'model']].copy()
result_df = pd.DataFrame()
factor_up_fac = 0.4
for m in [25,26]:
    tmp1 = df[m-3]*0.2 + df[m-1]*0.3 +df[m-2]*0.2 + df[m]*0.3
    tmp2 = df[m-12]
    tmp3 = df[m-24]
    temp_df['forecastVolum'] = (tmp1 + tmp2 + tmp3)/3 * (factor_up_fac*df['factor_up']+(1-factor_up_fac)*df['factor_down']) * 0.99
    temp_df['regMonth'] = m-24
    if m==26:
        temp_df['forecastVolum'] = temp_df['forecastVolum']*0.99*0.99
    result_df = result_df.append(temp_df, ignore_index=True)
    df[m] = temp_df['forecastVolum']
for m in [27,28]:
    tmp1 = df[m-3]/6 + df[m-1]/2 + df[m-2]/6 + (df[m-12]*0.6+df[m-24]*0.4)/6
    tmp2 = df[m-12]/2 + df[m-13]/4 + df[m-11]/4
    tmp3 = df[m-24]
    temp_df['forecastVolum'] = (tmp1 + tmp2 + tmp3)/3 * (factor_up_fac*df['factor_up']+(1-factor_up_fac)*df['factor_down']) * 0.99
    fac = 0.35*df['factor_up']+0.65*df['factor_down']
    temp_df['regMonth'] = m-24
    result_df = result_df.append(temp_df, ignore_index=True)
    df[m] = temp_df['forecastVolum']
new_data_test = pd.merge(new_data_test[['id', 'adcode', 'model', 'regMonth']], result_df, how='left', on=['adcode', 'model', 'regMonth'])
new_data_test['forecastVolum'] = np.exp(new_data_test['forecastVolum'])
print (new_data_test.groupby('regMonth')['forecastVolum'].mean())
new_data_test.loc[new_data_test['forecastVolum'] < 0, 'forecastVolum'] = 0

       adcode             model
36960  320000  bc5bbd459c18fefd
36988  320000  9011c3cf495e3c7b
37016  320000  1181894ae2811540
37044  320000  0aa15031db420212
37072  320000  283d331fdf05a655
...       ...               ...
50372  150000  2509a94e62fd3e4b
50400  150000  b135f55ced5c4c67
50428  150000  9a390098bf87b814
50456  150000  9e967965ad4a6c07
50484  150000  32d3069d17aa47c2

[484 rows x 2 columns]
regMonth
1    274.923193
2    172.246416
3    185.260309
4    181.364841
Name: forecastVolum, dtype: float64


In [7]:
new_data_test = pd.concat([old_data_test,new_data_test])
print (new_data_test.groupby('regMonth')['forecastVolum'].mean())
new_data_test[['id', 'forecastVolum']].round(0).astype(int).to_csv('submission_rule.csv', index=False)
print(new_data_test.head(5))


regMonth
1    546.104819
2    351.281438
3    378.497798
4    381.651866
Name: forecastVolum, dtype: float64
   id  adcode             model  regMonth  forecastVolum
0   1  310000  3c974920a76ac9c1         1     303.576973
1   2  530000  3c974920a76ac9c1         1     369.636611
2   3  150000  3c974920a76ac9c1         1     169.258498
3   4  110000  3c974920a76ac9c1         1     307.328115
4   5  510000  3c974920a76ac9c1         1     420.450773
